In [1]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.configs.vits_config import VitsConfig

from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
output_path = os.path.dirname(os.path.abspath('__file__'))


dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="Mixed_formatted.txt", path=os.path.join(output_path, "data/"), language = "en"
)

In [3]:
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

In [4]:
character_config = CharactersConfig(
    characters_class= "TTS.tts.models.vits.VitsCharacters",
    characters= "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890",
    punctuations=""" !,.?-"'""",
    pad= "<PAD>",
    eos= "<EOS>",
    bos= "<BOS>",
    blank= "<BLNK>",
)

In [5]:
config = VitsConfig(
    audio=audio_config,
    characters=character_config,
    run_name="vits_tyler1",
    batch_size=4,
    eval_batch_size=2,
    # batch_group_size=4,
    num_loader_workers=8,
    num_eval_loader_workers=2,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=5000,
    text_cleaner="english_cleaners",
    use_phonemes=False,
    # phoneme_language="en-us",
    # phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path + "/vitstts_checkpoint",
    datasets=[dataset_config],
    cudnn_benchmark=False,
    test_sentences = ["Mic test one two three", "I am Tyler1", "Welcome chat!"]
)

In [6]:
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "Tyler1"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.dirname(os.path.abspath('__file__')) + f"/data/wavs/{cols[0]}.wav"
            text = cols[1]
            # print(text)
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items

In [7]:
ap = AudioProcessor.init_from_config(config)

tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [8]:
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    # eval_split_max_size=config.eval_split_max_size,
    eval_split_size=0.05,
    formatter = formatter
)

 | > Found 60 files in /home/ubuntu/tyler1/data


In [9]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

In [10]:
trainer = Trainer(
    TrainerArgs(restore_path = output_path + "/vitstts_checkpoint/vits_tyler1-January-15-2024_08+30AM-dac07d8/best_model.pth"),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 30
 | > Num. of Torch Threads: 30
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
2024-01-15 12:35:34.530847: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 12:35:34.573366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
 > Start Tensorboard: tensorboard --logdir=/h

In [ ]:
trainer.fit()


 > EPOCH: 0/5000
 --> /home/ubuntu/tyler1/vitstts_checkpoint/vits_tyler1-January-15-2024_12+35PM-dac07d8

 > TRAINING (2024-01-15 12:35:37) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 57
 | > Preprocessing samples
 | > Max text length: 1462
 | > Min text length: 131
 | > Avg text length: 685.6140350877193
 | 
 | > Max audio length: 7408822.0
 | > Min audio length: 578318.0
 | > Avg audio length: 3487299.50877193
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
you look like lego piece two, fi wait. that is racist! that is racist as fuck bro! bro this guy typed in the chat you look like lego piece two five five zero c zero one' and look what it is. bro you are banned by the way. that is crazy. creative, but that's too much. that is crazy bro. fuck istwenty one. rx tylerone league players make fun of dadlerone_one
 [!] Character '_' not found in the vocabulary. Discarding it.


/home/ubuntu/.local/lib/python3.10/site-packages/torch/functional.py:650: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:863.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]

 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 3
 | > Preprocessing samples
 | > Max text length: 1148
 | > Min text length: 261
 | > Avg text length: 681.3333333333334
 | 
 | > Max audio length: 6162562.0
 | > Min audio length: 1429014.0
 | > Avg audio length: 3654345.3333333335
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


   --> STEP: 0
     | > loss_disc: 2.5209264755249023  (2.5209264755249023)
     | > loss_disc_real_0: 0.08970201015472412  (0.08970201015472412)
     | > loss_disc_real_1: 0.28978294134140015  (0.28978294134140015)
     | > loss_disc_real_2: 0.18989653885364532  (0.18989653885364532)
     | > loss_disc_real_3: 0.17149873077869415  (0.17149873077869415)
     | > loss_disc_real_4: 0.1891782134771347  (0.1891782134771347)
     | > loss_disc_real_5: 0.1630222648382187  (0.1630222648382187)
     | > loss_0: 2.5209264755249023  (2.5209264755249023)
     | > loss_gen: 1.7821269035339355  (1.7821269035339355)
     | > loss_kl: 1.1149191856384277  (1.1149191856384277)
     | > loss_feat: 4.246743679046631  (4.246743679046631)
     | > loss_mel: 19.20969009399414  (19.20969009399414)
     | > loss_duration: 2.330817937850952  (2.330817937850952)
     | > loss_1: 28.68429946899414  (28.68429946899414)

   --> STEP: 1
     | > loss_disc: 2.250843048095703  (2.250843048095703)
     | > loss_disc_r

...

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.005738258361816406 (-4.029273986816406e-05)
     | > avg_loss_disc: 2.3834285736083984 (-0.4828648567199707)
     | > avg_loss_disc_real_0: 0.0909360721707344 (-0.07781126350164413)
     | > avg_loss_disc_real_1: 0.35133078694343567 (+0.07216811180114746)
     | > avg_loss_disc_real_2: 0.11709269136190414 (-0.12153574079275131)
     | > avg_loss_disc_real_3: 0.2926129996776581 (-0.009837716817855835)
     | > avg_loss_disc_real_4: 0.31226494908332825 (-0.06540179252624512)
     | > avg_loss_disc_real_5: 0.3892958164215088 (-0.1135932207107544)
     | > avg_loss_0: 2.3834285736083984 (-0.4828648567199707)
     | > avg_loss_gen: 2.738616704940796 (+0.5160393714904785)
     | > avg_loss_kl: 1.051098346710205 (+0.1938711404800415)
     | > avg_loss_feat: 3.379382848739624 (+0.9017093181610107)
     | > avg_loss_mel: 28.678457260131836 (-0.5915164947509766)
     | > avg_loss_duration: 2.4284627437591553 (-0.00513768196105957)
     | > avg_

you look like lego piece two, fi wait. that is racist! that is racist as fuck bro! bro this guy typed in the chat you look like lego piece two five five zero c zero one' and look what it is. bro you are banned by the way. that is crazy. creative, but that's too much. that is crazy bro. fuck istwenty one. rx tylerone league players make fun of dadlerone_one
 [!] Character '_' not found in the vocabulary. Discarding it.



   --> TIME: 2024-01-15 14:56:33 -- STEP: 13/15 -- GLOBAL_STEP: 14025
     | > loss_disc: 2.2621428966522217  (2.3778371810913086)
     | > loss_disc_real_0: 0.07260115444660187  (0.13734741738209358)
     | > loss_disc_real_1: 0.27097055315971375  (0.21809731079981878)
     | > loss_disc_real_2: 0.29472482204437256  (0.2262379641716297)
     | > loss_disc_real_3: 0.24513059854507446  (0.2477686210320546)
     | > loss_disc_real_4: 0.20510776340961456  (0.19749360015759101)
     | > loss_disc_real_5: 0.2122095227241516  (0.24528407362791207)
     | > loss_0: 2.2621428966522217  (2.3778371810913086)
     | > grad_norm_0: tensor(24.2356, device='cuda:0')  (tensor(28.4157, device='cuda:0'))
     | > loss_gen: 2.874507427215576  (2.4421405058640695)
     | > loss_kl: 0.8783648610115051  (0.9582773538736197)
     | > loss_feat: 6.389960289001465  (5.222638313586895)
     | > loss_mel: 30.05840301513672  (30.01254477867713)
     | > loss_duration: 2.3408522605895996  (2.3641484700716457)
  

 | > Synthesizing test sentences.
